In [ ]:
from math import log10


In [202]:
from ctypes import c_uint, c_float

def cast(num, rettype):
    """
    Given a ctypes type, it attempts to read read a number from 
    some memory type and interpret it as a different type
    This will have overhead, but will provide a cast like interface
    """
    return rettype.from_buffer(num).value

def cast_int_as_float(n):
    return cast(c_uint(n), c_float)

def cast_to_float(n):
    return cast(c_uint(n), c_float)

def cast_float_as_int(n):
    return cast(c_float(n), c_uint)

def cast_to_int(n):
    return cast(c_float(n), c_uint)


In [182]:
def fast_inv_sqrt(num):
    halfnum = 0.5 * num
    # take the number, make it a float and read it into an integer
    inum = cast_float_as_int(num)
    # use the hack
    hack = 0x5f3759df - (inum >> 1)
    # back into floating point
    fnum = cast_int_as_float(hack)
    print(f"guess {fnum}")
    return fnum * (1.5 - halfnum * fnum * fnum) 
    
def inv_sqrt(num):
    return 1/num**(0.5)

In [177]:
test_num = 250
(fast_inv_sqrt(test_num) - inv_sqrt(test_num))**2

1.1207861519762848e-08

# Floating Point Numbers

For the sake of simplicity, let us look at 32 bit floats. A 32 bit float is divided into 3 sections:

1. The first bit for the sign. 1 in this location represents a negative number
2. The next 8 bits for the exponent. This uses unsigned representation, but is normalised by subtracting -127 to get the floating point ranges. Note, this is not using 2's complement for it's representation. 1 in this space represents -126 and 0x80 represents 1 for example. (It's effectively a negative 2's complement)
3. The remaining 23 bits for the mantissa. This is represented in fixed point with 1 to be added. I.e. the leading bit represents 0.5 to which 1 is added for the mantissa expression. 

So a floating point is given by by
`sign * (1 + mantissa) * 2 ^ exponent`

There is a special denormalised mode, when the exponent is 0, the exponent is treated as -126 and the 1+mantissa becomes just mantissa.

This link here: https://www.h-schmidt.net/FloatConverter/IEEE754.html should give you some additional insight. And Wikipedia: https://en.wikipedia.org/wiki/Single-precision_floating-point_format explains it in depth too

Some of this should explain things we've seen using floating point

1. Since the mantissa is fixed point, the (1+mantissa) is effectively from 1 to 2
2. That the largest number is 2 * 2 ^ 127 = 2^ 128 ~ 3.4 * 10 ^ 38
3. Smallest positive is obtained from the denormalised mode with is 2^-126 * 2 ^-23 = 2 ^ -149 ~ 7.0 x 10^-46
4. Smallest change in an arbitrary floating point is given by 2 ^ -23 ~ 1.19 * 10 ^-7

# Fast Inverse Square Root

The Fast inverse square root is just Newton's method with a good initial guess. Follow the maths here for some more information: https://betterexplained.com/articles/understanding-quakes-fast-inverse-square-root/. 


# The Initial guess

This line`0x5f3759df - (inum >> 1)` is the secret sauce above, getting a good approximation. Let's break this down.

Noting that the exponent is a bit out of sync with the mantissa, let's rewrite the magic number slightly

1. Exponent = 0xbe = 189
2. Mantissa = 0x3759df

In [178]:
hex(0x5f << 1)

'0xbe'

## The exponent

Let's simplify this to a problem which is purely a power of 2. Let's consider `2^4` and how we would use bit manipulation to get `2^-2` as the answer. We're going to deal with odd powers later (since we need to manipulate the mantissa too)


Noting that we want a good estimate of the inverse square root, when provided a number in exponent form, we instantly know that halving and inverting the exponent (i.e. making 10 ^ 4 into 10 ^ -2) is a good idea. So that's what we do.

For a simple unsigned integer, doing a right shift is enough to halve it's value, i.e. n >> 1. Again, remembering that the exponent is an unsigned 8 bit integer with a -127 offset. To halve a value stored in the exponent form we need to do  
`exponent/2 = (exponent - 127) >> 1  + 127`

And using this logic, to get the negative half  
`-exponent/2 = 127 - ((exponent - 127) >> 1)`

Expanding the above we get

`127 - (exponent -126 -1) >> 1`  
`127 + 63 - (exponent -1) >> 1`  
`190 - (exponent -1) >> 1`  

Depending on whether we have an odd or even exponent we need to decide on the result of the above. Note even exponents, such as `2^4` would have the float exponent set to `131`. So the above simplifies to 

`190 - (exponent >> 1)`

This 190 is 1 off from the 189 above and we'll come across why later

Note that 190 is `0xbf` and we represent this mask as `0x5f800000` after we shift it to the right to make space for the sign bit. 

In [241]:
hex(0xbf0 >> 1)

'0x5f8'

In [203]:
cast_to_float(0x5f800000 - (cast_to_int(64.0) >> 1))

0.1875

In [201]:
1/8

0.125

We're not quite there, what happened? Well, when we have floating point numbers, our shift from the exponent falls into the mantissa. This makes our mantissa from 1 to 1.5. When we have an odd exponent, we don't have this scale factor happen and instead get floor of the exponent halved.

In [204]:
1/8 * 1.5

0.1875

In [244]:
cast_to_float(0x5f800000 - (cast_to_int(32.0) >> 1))

0.25

To fix this, we can change the 8 to 4 to catch the overflow. 

In [242]:
cast_to_float(0x5f800000 - (cast_to_int(32.0) >> 1))

0.25

In [238]:
cast_to_float(0x5f400000 - (cast_to_int(16) >> 1))

0.25

In [239]:
cast_to_float(0x5f400000 - (cast_to_int(0.25) >> 1))

2.0

This doesn't work for 

In [248]:
cast_to_float(0x5f300000 - (cast_to_int(32) >> 1))

0.171875